# Qdrant Vector Database Demo

## Overview
This notebook demonstrates how to:
- Set up and run Qdrant using Docker
- Connect to Qdrant using the Python client
- Create collections for vector storage
- Upload and search vector data
- Perform basic vector database operations

## Prerequisites
- Docker installed and running
- Python environment with `qdrant-client` package
- Basic understanding of vector databases

---


## Step 1: Docker Setup

### Running Qdrant with Docker

Before we can use Qdrant, we need to start the Qdrant server using Docker. Run the following command in your terminal:

```bash
docker run -p 6333:6333 -p 6334:6334 \
    -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
    qdrant/qdrant
```

**What this command does:**
- `-p 6333:6333`: Maps port 6333 (HTTP/REST API) from container to host
- `-p 6334:6334`: Maps port 6334 (gRPC API) from container to host  
- `-v "$(pwd)/qdrant_storage:/qdrant/storage:z"`: Mounts a local directory for persistent storage
- `qdrant/qdrant`: Uses the official Qdrant Docker image

**Expected output:**
- Qdrant logo and version information
- Web UI available at: http://localhost:6333/dashboard
- Server listening on ports 6333 and 6334

---


## Step 2: Python Client Setup

### Installing the Qdrant Client

First, make sure you have the `qdrant-client` package installed in your environment:

```bash
# Using pip
pip install qdrant-client

# Using uv (recommended for this project)
uv add qdrant-client
```

### Importing and Connecting to Qdrant

Now let's import the client and establish a connection to our running Qdrant instance.


In [ ]:
# Import the Qdrant client
from qdrant_client import QdrantClient

# Create a client connection to the local Qdrant instance
# Port 6333 is the HTTP/REST API endpoint
client = QdrantClient(url="http://localhost:6333")

# Test the connection by getting server info
try:
    collections = client.get_collections()
    print("✅ Successfully connected to Qdrant!")
    print(f"📊 Current collections: {len(collections.collections)}")
    print(f"🔗 Server URL: http://localhost:6333")
    print(f"🌐 Web UI: http://localhost:6333/dashboard")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("Make sure Qdrant is running with: docker run -p 6333:6333 qdrant/qdrant")


In [34]:

client.create_collection(
    collection_name='my_collection',
    vectors_config={
        "size":384,
        "distance":"Cosine"
    }    )

True

In [35]:
import numpy as np

# Create simple sample data
points = [
    {
        "id": 1,
        "vector": np.random.randn(384).tolist(),  # Random 384-dim vector
        "payload": {"text": "Machine learning is awesome", "category": "AI"}
    },
    {
        "id": 2,
        "vector": np.random.randn(384).tolist(),  # Random 384-dim vector
        "payload": {"text": "Python is great for data science", "category": "Programming"}
    }
]

# Upload points to collection
client.upsert(collection_name='my_collection', points=points)
print(f"✅ Uploaded {len(points)} points to collection")


✅ Uploaded 2 points to collection


## Step 5: Search


In [37]:
# Create query vector and search
query_vector = np.random.randn(384).tolist()
results = client.search(
    collection_name='my_collection',
    query_vector=query_vector,
    limit=2
)

# Display results
for result in results:
    print(f"Score: {result.score:.3f} | Text: {result.payload['text']}")


Score: 0.040 | Text: Python is great for data science
Score: -0.074 | Text: Machine learning is awesome


/var/folders/p_/53rzshtx2lb_c4blf251m6d40000gn/T/ipykernel_23309/3027129300.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [ ]:
#Checking how the values are stored
